In [3]:
import os
import pandas as pd
import scanpy as sc
import os
import numpy as np
import anndata as ad
from anndata import AnnData
from anndata import read_h5ad
import gc  

DATASET_DIR = "/work/upcourtine/clock-classifier/gabriele-results/census_results/datasets"

In [2]:
# Find paths of all datasets
dataset_paths = []
for file_name in os.listdir(DATASET_DIR):
    if file_name.endswith('.h5ad'):
        file_path = os.path.join(DATASET_DIR, file_name)
        if os.path.isfile(file_path):  # Check if it's a file
            dataset_paths.append(file_path)

# Columns of .obs that each df must have
must_columns = ["donor_id", 
                "cell_type", "cell_type_ontology_term_id",
                "disease", "disease_ontology_term_id", 
                "tissue", "tissue_ontology_term_id",
                "assay", "assay_ontology_term_id"]

In [ ]:
# check the structre of a single obekct
a = dataset_paths[0]
print(a)
adata = sc.read(a)
print(adata)
display(adata.obs.columns)
display(adata.obs.head(2))

# Check that all necessary columns are in all df

In [ ]:
# Checj that each df has the "must_columns"
for dataset in dataset_paths:
    print(dataset)

    # Read the .h5ad file using scanpy
    adata = sc.read(dataset)

    # Check that the necessary columns are present
    missing_columns = [col for col in must_columns if col not in adata.obs.columns]
    if missing_columns:
        print(f"Missing the following required columns: {', '.join(missing_columns)}\n")


    # just to make some checks 
    if (adata.obs["organism"] != "Homo sapiens").any():
        print("Non-humans cells detected")
    if (adata.obs["is_primary_data"] == False).any():
        print("Non Primary cells detected")


# Merge Datasets

- Maintain ONLY genes in `gene_to_keep`

- Remove useless info (ex, pca emb)

- If a gene is missing add it and put all its values to 0.

In [3]:
# Read mapping file
# ATTENTION: if not yet created see last part of the notebook
df_genes_map = pd.read_csv( os.path.join("/home/dalai/git/clock-classifier/Gabriele/assets", "mapping_genes_census.csv"), index_col = False)
df_genes_map.head(2)

,gene_id,gene_name
0,ENSG00000290825,DDX11L16
1,ENSG00000223972,DDX11L1


In [4]:
# `gene_to_keep` is the list of genes you want to keep --> genes in all datasets
gene_to_keep = df_genes_map["gene_id"]
gene_to_keep

0        ENSG00000290825
1        ENSG00000223972
2        ENSG00000310526
3        ENSG00000227232
4        ENSG00000278267
              ...       
86397    ENSG00000303867
86398    ENSG00000303902
86399    ENSG00000306528
86400    ENSG00000297844
86401    ENSG00000309258
Name: gene_id, Length: 86402, dtype: object

## Remove Useless Info

In [5]:
adata_datasets_list = []

# Remove genes not in list
# + 
# Remove useless information
for i, dataset in enumerate(dataset_paths):
    
    print(f"\n{dataset}")
    print(f"Dataset {i+1}/{len(dataset_paths)}")
    
    # Read AnnData
    adata = sc.read(dataset)

    #display(adata.var)

    # Extract the genes in the AnnData object
    genes_in_adata = adata.var.index.tolist()

    # Identify genes in the AnnData object that are not in the list
    genes_to_remove = [gene for gene in genes_in_adata if gene not in gene_to_keep]

    # Remove the genes that are not in the list
    adata = adata[:, ~adata.var.index.isin(genes_to_remove)]


    # Shrink the dataset, i.e. mantain only
        # raw count matrix
        # .var
        # subset of .obs

    # Retain only the necessary columns in `obs`
    adata.obs = adata.obs[must_columns]

    # Remove all keys from `uns` (unstructured annotations)
    adata.uns = {}

    # Remove all keys from `obsm` (observations in multi-dimensional space)
    adata.obsm = {}

    adata_datasets_list.append(adata)

    # Clear memory for the next iteration
    del adata
    gc.collect()




/work/upcourtine/clock-classifier/gabriele-results/census_results/datasets/Leng_2021_1.h5ad
Dataset 1/30

/work/upcourtine/clock-classifier/gabriele-results/census_results/datasets/Fenton_2021_1.h5ad
Dataset 2/30

/work/upcourtine/clock-classifier/gabriele-results/census_results/datasets/Jardine_2021_1.h5ad
Dataset 3/30

/work/upcourtine/clock-classifier/gabriele-results/census_results/datasets/Jäkel_2019_1.h5ad
Dataset 4/30

/work/upcourtine/clock-classifier/gabriele-results/census_results/datasets/King_2021_1.h5ad
Dataset 5/30

/work/upcourtine/clock-classifier/gabriele-results/census_results/datasets/Kuppe_2022_1.h5ad
Dataset 6/30

/work/upcourtine/clock-classifier/gabriele-results/census_results/datasets/Triana_2021_1.h5ad
Dataset 7/30

/work/upcourtine/clock-classifier/gabriele-results/census_results/datasets/Hernández_2022_1.h5ad
Dataset 8/30

/work/upcourtine/clock-classifier/gabriele-results/census_results/datasets/Smith_2021_1.h5ad
Dataset 9/30

/work/upcourtine/clock-classif

In [6]:
len(adata_datasets_list)

30

## Merge + Assure all genes are inside

In [7]:
# Merge all datasets
# +
# Add missing expression values for mssing genes for each dataset
    # easy: if we merge using "outer", by defiuakt missing genes will be put to 0

# Create an empty AnnData object with all genes from df_genes_map
all_genes_anndata = AnnData(
    X=np.zeros((0, df_genes_map.shape[0])),  # 0 "dummy" observation with zeros for all genes
    var=pd.DataFrame(index=df_genes_map["gene_id"])  # Set var to have all genes from df_genes_map
)

# Ensure all genes in df_genes_map are added
print(f"All genes AnnData object: {all_genes_anndata}")

# Append the empty AnnData object to the list of datasets
adata_datasets_list.append(all_genes_anndata)

# Merge all datasets using `outer` join to include all genes
big_adata = ad.concat(adata_datasets_list, join="outer", label="orig_dataset", fill_value=0)

display(big_adata.obs)
display(big_adata.var)
print(big_adata.X.shape)


All genes AnnData object: AnnData object with n_obs × n_vars = 0 × 86402


/home/dalai/miniconda3/envs/census_env/lib/python3.10/site-packages/anndata/_core/merge.py:1358: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(
/home/dalai/miniconda3/envs/census_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


,donor_id,cell_type,cell_type_ontology_term_id,disease,disease_ontology_term_id,tissue,tissue_ontology_term_id,assay,assay_ontology_term_id,orig_dataset
EC2_AAACCTGAGGATGCGT,2,glutamatergic neuron,CL:0000679,normal,PATO:0000461,entorhinal cortex,UBERON:0002728,10x 3' v2,EFO:0009899,0
EC2_AAACCTGAGTCAATAG,2,glutamatergic neuron,CL:0000679,normal,PATO:0000461,entorhinal cortex,UBERON:0002728,10x 3' v2,EFO:0009899,0
EC2_AAACCTGCATACTCTT,2,glutamatergic neuron,CL:0000679,normal,PATO:0000461,entorhinal cortex,UBERON:0002728,10x 3' v2,EFO:0009899,0
EC2_AAACCTGGTCCAACTA,2,glutamatergic neuron,CL:0000679,normal,PATO:0000461,entorhinal cortex,UBERON:0002728,10x 3' v2,EFO:0009899,0
EC2_AAACCTGGTTATCACG,2,oligodendrocyte precursor cell,CL:0002453,normal,PATO:0000461,entorhinal cortex,UBERON:0002728,10x 3' v2,EFO:0009899,0
...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTAGGACAC-1-208,pat.16,alpha-beta T cell,CL:0000789,Crohn ileitis,MONDO:0000709,ileum,UBERON:0002116,10x 3' v2,EFO:0009899,29
TTTGTCAGTCGGATCC-1-208,pat.16,alpha-beta T cell,CL:0000789,Crohn ileitis,MONDO:0000709,ileum,UBERON:0002116,10x 3' v2,EFO:0009899,29
TTTGTCAGTGCGATAG-1-208,pat.16,alpha-beta T cell,CL:0000789,Crohn ileitis,MONDO:0000709,ileum,UBERON:0002116,10x 3' v2,EFO:0009899,29
TTTGTCATCATTGCCC-1-208,pat.16,alpha-beta T cell,CL:0000789,Crohn ileitis,MONDO:0000709,ileum,UBERON:0002116,10x 3' v2,EFO:0009899,29


""
ENSG00000290825
ENSG00000223972
ENSG00000310526
ENSG00000227232
ENSG00000278267
...
ENSG00000303867
ENSG00000303902
ENSG00000306528
ENSG00000297844


(465679, 86402)


## Add gene_name in .var

In [8]:
# Add colun in .var with gene name

# Create a mapping from Ensembl IDs to Gene Names
name_mapping = dict(zip(df_genes_map['gene_id'], df_genes_map['gene_name']))

big_adata.var['gene_names'] = big_adata.var_names.map(name_mapping)

big_adata.var

,gene_names
ENSG00000290825,DDX11L16
ENSG00000223972,DDX11L1
ENSG00000310526,WASH7P
ENSG00000227232,WASH7P
ENSG00000278267,MIR6859-1
...,...
ENSG00000303867,ENSG00000303867
ENSG00000303902,ENSG00000303902
ENSG00000306528,ENSG00000306528
ENSG00000297844,ENSG00000297844


## Remove cells with duplicated barcode

In [9]:
# Remove cells that now have same barcode

duplicates = big_adata.obs_names[big_adata.obs_names.duplicated()]
print(duplicates)

big_adata.obs_names_make_unique() #modifies in place

duplicates = big_adata.obs_names[big_adata.obs_names.duplicated()]
print(duplicates)


Index(['AAGCCGCAGTGGTCCC-1', 'ACCAGTACACGACGAA-1', 'ACGATACTCGCGATCG-1',
       'ACGGCCAAGTGACATA-1', 'ATAACGCGTGCATCTA-1', 'CAAGATCGTTAAAGAC-1',
       'CACAGGCCACACCGCA-1', 'CAGCAGCGTGGCTCCA-1', 'CAGTCCTCATGACATC-1',
       'CCGTACTTCAACACGT-1',
       ...
       'CCCAATCGTCCTGCTT-5', 'TTTGCGCCAGTCGATT-5', 'AGAATAGAGCACCGTC-5',
       'CGTGAGCAGTGATCGG-5', 'GGAAAGCCAATCCAAC-5', 'CGGTTAATCACTTATC-5',
       'AGTCTTTCATCCCACT-5', 'TCATTTGGTGCGGTAA-5', 'AAGGCAGTCTGCCAGG-5',
       'CAGCCGAAGAAACCTA-5'],
      dtype='object', length=176)
Index([], dtype='object')


## Normalize Expression ???

In [ ]:
# TODO

## Save

In [10]:
# Save
print(big_adata)

big_adata.write("/work/upcourtine/clock-classifier/gabriele-results/census_results/merged_30.h5ad")


AnnData object with n_obs × n_vars = 465679 × 86402
    obs: 'donor_id', 'cell_type', 'cell_type_ontology_term_id', 'disease', 'disease_ontology_term_id', 'tissue', 'tissue_ontology_term_id', 'assay', 'assay_ontology_term_id', 'orig_dataset'
    var: 'gene_names'


# Further Refinement

In [4]:
big_adata = read_h5ad("/work/upcourtine/clock-classifier/gabriele-results/census_results/merged_30.h5ad")
big_adata

AnnData object with n_obs × n_vars = 465679 × 86402
    obs: 'donor_id', 'cell_type', 'cell_type_ontology_term_id', 'disease', 'disease_ontology_term_id', 'tissue', 'tissue_ontology_term_id', 'assay', 'assay_ontology_term_id', 'orig_dataset'
    var: 'gene_names'

In [5]:
big_adata.obs.head(5)


,donor_id,cell_type,cell_type_ontology_term_id,disease,disease_ontology_term_id,tissue,tissue_ontology_term_id,assay,assay_ontology_term_id,orig_dataset
EC2_AAACCTGAGGATGCGT,2,glutamatergic neuron,CL:0000679,normal,PATO:0000461,entorhinal cortex,UBERON:0002728,10x 3' v2,EFO:0009899,0
EC2_AAACCTGAGTCAATAG,2,glutamatergic neuron,CL:0000679,normal,PATO:0000461,entorhinal cortex,UBERON:0002728,10x 3' v2,EFO:0009899,0
EC2_AAACCTGCATACTCTT,2,glutamatergic neuron,CL:0000679,normal,PATO:0000461,entorhinal cortex,UBERON:0002728,10x 3' v2,EFO:0009899,0
EC2_AAACCTGGTCCAACTA,2,glutamatergic neuron,CL:0000679,normal,PATO:0000461,entorhinal cortex,UBERON:0002728,10x 3' v2,EFO:0009899,0
EC2_AAACCTGGTTATCACG,2,oligodendrocyte precursor cell,CL:0002453,normal,PATO:0000461,entorhinal cortex,UBERON:0002728,10x 3' v2,EFO:0009899,0


## Create Labels

They are a concatenation of cell type and disease.

In [7]:
big_adata.obs["concat_label"] = big_adata.obs["cell_type_ontology_term_id"].astype(str) + '-' + big_adata.obs["disease_ontology_term_id"].astype(str)
print("In total the differtn labels are: ", big_adata.obs["concat_label"].unique().shape)

# Encode Labels as int
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
big_adata.obs["concat_label_encoded"] = label_encoder.fit_transform(big_adata.obs["concat_label"])

big_adata.obs["concat_label_encoded"].sort_values()

In total the differtn labels are:  (265,)


p51c_CTAGTGACACTTCGAA          0
p51c_GAAATGAAGTTGCAGG          0
p51c_GCGCGATCACTACAGT          0
p51c_GTAGGCCCAAGTTAAG          0
p51c_TACACGAAGGTCATCT          0
                            ... 
n.1.1_AATCGTGCAATCTCGA       264
n.1.1_AATGAAGGTCTACATG       264
GSM2172193_1000010014.B12    264
GSM2172229_1000010015.A10    264
GSM2173997_1000102504.H11    264
Name: concat_label_encoded, Length: 465679, dtype: int64

In [8]:
# Save
big_adata.write("/work/upcourtine/clock-classifier/gabriele-results/census_results/merged_30_labelled.h5ad")


# *****************************************************

# Make GeneMapping File

Raw data from [here](https://www.gencodegenes.org/human/).

Unzip the file `gunzip gencode.v47.chr_patch_hapl_scaff.annotation.gtf.gz`.

In [ ]:
import pandas as pd
import re

# Define chunk size
chunk_size = 100000

# Initialize an empty list to store chunks
chunks = []

# Read the GTF file in chunks
for chunk in pd.read_csv("/home/dalai/git/clock-classifier/Gabriele/assets/gencode.v47.chr_patch_hapl_scaff.annotation.gtf", 
                         sep="\t", 
                         comment="#", 
                         header=None, 
                         names=["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attributes"], 
                         chunksize=chunk_size):
    
    # Extract gene_id, transcript_id, gene_name, and gene_type from the 'attributes' column using regular expressions
    chunk['gene_id'] = chunk['attributes'].apply(lambda x: re.search(r'gene_id "([^"]+)"', x).group(1) if re.search(r'gene_id "([^"]+)"', x) else None)
    chunk['transcript_id'] = chunk['attributes'].apply(lambda x: re.search(r'transcript_id "([^"]+)"', x).group(1) if re.search(r'transcript_id "([^"]+)"', x) else None)
    chunk['gene_name'] = chunk['attributes'].apply(lambda x: re.search(r'gene_name "([^"]+)"', x).group(1) if re.search(r'gene_name "([^"]+)"', x) else None)
    chunk['gene_type'] = chunk['attributes'].apply(lambda x: re.search(r'gene_type "([^"]+)"', x).group(1) if re.search(r'gene_type "([^"]+)"', x) else None)

    # Add the chunk to the list
    chunks.append(chunk)

# Combine the chunks into a single DataFrame
df = pd.concat(chunks, ignore_index=True)

# Display the first few rows with gene_id, transcript_id, gene_name, and gene_type
display(df[['seqname', 'feature', 'start', 'end', 'gene_id', 'transcript_id', 'gene_name', 'gene_type']].head())


In [ ]:
print(df.feature.unique())
print(df.gene_type.unique())


In [ ]:
# Filter

print(df.shape)
print(df[df.feature == "gene"].shape)

# Take only elements that are genes
print(df.feature.unique())
df = df[df.feature == "gene"]

# Maybe filter more based on "gene_type" ??
# TODO

df.head(4)

In [ ]:
# Take only mapping colums
df = df[["gene_id", "gene_name"]]
df.head(5)

In [ ]:
# Remove "version" from ensamble id

print(df['gene_id'].str.split('.').str[0].size) #beforee
print(df['gene_id'].str.split('.').str[0].unique().size) #see if thwere are some duplicates after removing version

df['gene_id'] = df['gene_id'].str.split('.').str[0]


In [ ]:
# Save
save_mapping_df_path = os.path.join("/home/dalai/git/clock-classifier/Gabriele/assets", "mapping_genes_census.csv")
df.to_csv(save_mapping_df_path, index=False)


In [ ]:
# little chck
df = pd.read_csv(save_mapping_df_path, index_col = False)
df